In [3]:
import os
import dill as dpickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from ktext.preprocess import processor

C:\tools\Anaconda3\envs\python3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [5]:
cwd = os.getcwd()
print(cwd)

C:\Users\hagrawa2\Desktop\issues_labeller


In [7]:
df = pd.concat([pd.read_csv("{}\issues_data_00000000000{}.csv".format(cwd, i)) for i in range(0, 10)])

In [8]:
df.head()

,url,repo,title,body,num_labels,labels,c_bug,c_feature,c_question,c_documentation,class_int
0,"""https://github.com/dontsovcmc/waterius/issues...",dontsovcmc/waterius,передавать кол-во перезагрузок,1. так все делают\r 2. будет полезно для изуче...,1,"[""enhancement"", ""enhancement""]",False,True,False,False,1
1,"""https://github.com/IlyaPetrovM/razmetka/issue...",IlyaPetrovM/razmetka,синхронное перемещение интервалов текста,нужно переделать систему сигналов. ! проблема...,2,"[""enhancement"", ""help wanted""]",False,True,False,False,1
2,"""https://github.com/Microsoft/vscode-cpptools/...",Microsoft/vscode-cpptools,“format codes” is unusable,i can't use \ format codes\ function both in ...,2,"[""bug"", ""Language Service""]",True,False,False,False,0
3,"""https://github.com/vnpy/vnpy/issues/1029""",vnpy/vnpy,一直报 no module named websocket 安装也安装不了,环境\r \r 操作系统: ubuntu 16.04 （阿里云）\r anacond...,1,"[""question""]",False,False,True,False,2
4,"""https://github.com/superbiger/pinyin4js/issue...",superbiger/pinyin4js,下载下来 npm run build-browser 报不成功,npm run build-browser\r \r error in pinyin4js....,1,"[""question""]",False,False,True,False,2


In [9]:
df.shape

(2201347, 11)

In [10]:
traindf, testdf = train_test_split(df, test_size=.15)
traindf.to_pickle('traindf.pkl')
testdf.to_pickle('testdf.pkl')

In [12]:
traindf.groupby('class_int').size()

class_int
0    834610
1    855858
2    180676
dtype: int64

In [13]:
pareto_df = pd.DataFrame({'pcnt': df.groupby('repo').size() / len(df), 'count': df.groupby('repo').size()})
pareto_df.sort_values('pcnt', ascending=False).head(20)

,pcnt,count
repo,,
Microsoft/vscode,0.005305,11679
MicrosoftDocs/azure-docs,0.002831,6232
rancher/rancher,0.002393,5267
ansible/ansible,0.002313,5091
godotengine/godot,0.002176,4791
kubernetes/kubernetes,0.002172,4781
hashicorp/terraform,0.001485,3269
elastic/kibana,0.001446,3184
eclipse/che,0.001311,2885


In [14]:
%%time

train_body_raw = traindf.body.tolist()
train_title_raw = traindf.title.tolist()

# Clean, tokenize, and apply padding / truncating such that each document length = 75th percentile for the dataset.
#  also, retain only the top keep_n words in the vocabulary and set the remaining words
#  to 1 which will become common index for rare words 

# process the issue body data
body_pp = processor(heuristic_pct_padding=.75, keep_n=8000)
train_body_vecs = body_pp.fit_transform(train_body_raw)

# process the title data
title_pp = processor(heuristic_pct_padding=.75, keep_n=4500)
train_title_vecs = title_pp.fit_transform(train_title_raw)

 See full histogram by insepecting the `document_length_stats` attribute.
 See full histogram by insepecting the `document_length_stats` attribute.


Wall time: 8min 46s


In [15]:
%%time

test_body_raw = testdf.body.tolist()
test_title_raw = testdf.title.tolist()

test_body_vecs = body_pp.transform_parallel(test_body_raw)
test_title_vecs = title_pp.transform_parallel(test_title_raw)

Wall time: 1min 34s


In [16]:
train_labels = np.expand_dims(traindf.class_int.values, -1)
test_labels = np.expand_dims(testdf.class_int.values, -1)

In [17]:
# Save the preprocessor
with open('body_pp.dpkl', 'wb') as f:
    dpickle.dump(body_pp, f)

with open('title_pp.dpkl', 'wb') as f:
    dpickle.dump(title_pp, f)

# Save the processed data
np.save('train_title_vecs.npy', train_title_vecs)
np.save('train_body_vecs.npy', train_body_vecs)
np.save('test_body_vecs.npy', test_body_vecs)
np.save('test_title_vecs.npy', test_title_vecs)
np.save('train_labels.npy', train_labels)
np.save('test_labels.npy', test_labels)

In [19]:
assert train_body_vecs.shape[0] == train_title_vecs.shape[0] == train_labels.shape[0]
assert test_body_vecs.shape[0] == test_title_vecs.shape[0] == test_labels.shape[0]